**Table of contents**

* [Language models](#lm)
* [Unigram LMs](#unigram)
* [Smoothing](#smooth)
* [Higher-Order LMs](#higher)
* [Interpolation](#inter)
* [Evaluation](#eval)  
* [Tuning hyperparameters](#hparams)
    
**Table of Exercises**

* [Exercise 1](#ex1) (-/3)
* [Exercise 2](#ex2) (-/2)
* [Exercise 3](#ex3) (-/1)
* [Exercise 4](#ex4) (-/1)
* [Exercise 5](#ex5) (-/3)

**General notes**

* In this notebook you are expected to use $\LaTeX$. 
* Use python3

**ILOs**

After completing this lab you should be able to 

* develop ngram language models
* estimate parameters of LMs via MLE
* evaluate LMs intrinsically in terms of perplexity

# <a name="lm"> Language Models
    
As we began discussing in [T1](../T1/T1.ipynb), a language model (LM) is a **probability distribution over text**, where text is a finite sequences of words. 
    
LMs can be used to generate text as well as to quantify a degree of naturalness (or rather, a degree of resemblance to training data) which is useful for example to compare and rank alternative pieces of text in terms of fluency.
    
To design an LM, we need to talk about units of text (e.g., documents, paragraphs, sentences) as outcomes of a random experiment, and for that we need *random variables*. If you need a recap on probability theory, don't miss out [our refresher](../R1/R1.ipynb).
    
We will generally refer to the unit of text as a *sentence*, but that's just for convenience, you could design an LM over documents and very little, if anything, would change.  

A **random sentence** is a finite sequence of tokens in the vocabulary of a given language. As a running example, we will refer to this language as English. The vocabulary of English will be denoted by $\Sigma$, a finite collection of unique tokens (which, as we discussed in T1, may include words, punctuation, special symbols, etc). We will denote a random sentence by $S$, or more explicitly, by the random sequence $X_{1:m} = \langle X_1, \ldots, X_m \rangle$. Here $m$ indicates the sequence length. Each token in the sequence is a random variable that takes on values in $\mathcal X$, an enumeration of $\Sigma$. We will adopt an important convention, every sentence is a finite sequence that ends with a special symbol, the end-of-sequence (EOS) symbol. Clearly, random sentences take on values in the set $\Sigma^*$ of all strings made of symbols in $\Sigma$, which is a set that does include an infinte number of valid English sentences (possibly not all English sentences, as our vocabulary may not be complete enough, but hopefully this space is still large enough for the LM to be useful) as well as an infinite number of sequences that are not valid English sentences.       
    
Part of the deal with a language model is to define and estimate a probability distribution that expresses a preference for sentences that are more likely to be accepted as English sentences. In practice an LM will prefer sentences that reproduce statistics of the observations used to estimate its parameters, whether these sentences will resemble English sentences or not will depend on how expressive the LM is, that is, whether or not the LM can capture patterns as complex as those arising from well-formed English (or whatever variant/register of English was observed during training).
    
**Notation guide** Some textbooks or papers use $X_1^m$ instead of $X_{1:m}$ for ordered sequences, both are clear enough, but we will use the notation adopted by the textbook, that is, $X_{1:m}$. The textbook uses $X_1\cdots X_m$ (without commas) as another notation for ordered sequences, but we prefer to explicitly mark the sequence with angle brackets to avoid ambiguities, i.e., we prefer $\langle X_1, \ldots, X_m \rangle$. The textbook is not very formal with respect to probability distribution notation, we prefer to be more formal about it, so we will subscript $P$ with the random variable(s) of interest, e.g., $P_{X_1:m}$ for a distribution over sequences, or $P_X$ for a distribution over words, or $P_{X|H}$ for a conditional distribution.

**Quiz 1** Define a categorical random variable $X$ for words sampled from a closed vocabulary $\Sigma$ (assume the size of the vocabulary is denoted by $v$). In your answer make sure you indicate what is the sample space and the precise support $\mathcal X$ of the categorical random variable. 

<details>
    <summary><b>SOLUTION</b></summary>

Let $X$ be a random variable that maps from a vocabulary $\Sigma$ to an index set of $\Sigma$ (the support) via an enumeration. The sample space is $\Sigma$, that is, the words in the language. The support of the random variable is the enumeration, that is the set of integers from 1 to $v$, i.e. $\mathcal X = \{1, \ldots, v\}$.

</details>

---

The following notational shortcuts are rather convenient:

* we will often use $X_{1:m}$ for a random sentence instead of the longer form $\langle X_1, \ldots, X_m \rangle$, and similarly for outcomes (i.e., $x_{1:m}$ instead of $\langle x_1, \ldots, x_m\rangle$), but in the long form we shall never drop the angle brackets, as otherwise it's hard to tell that we mean an ordered sequence
* we will use $X_{<i}$ (or $x_{<i}$ for an outcome) to denote the sequence of tokens that precedes the $i$th token, this sequence is empty $X_{<i} \triangleq \langle \rangle$ for $i \le 1$, for $i>1$ the sequence is defined as $X_{<i} \triangleq \langle X_1, \ldots, X_{i-1}\rangle$
* sometimes it will be useful to find a more compact notation for $X_{<i}$, in those cases we refer to it as a *random history* and denote it by $H_i$.



LMs can be described by a **generative story**, that is, a stochastic procedure that explains how an outcome $x_{1:m}$ is drawn from the model distribution. Though we may find inspiration in how we believe the data were generated, the generative story is not a faithful representation of any linguistic process, it is all but an abstraction that codes our own assumptions about the problem. 

The most general form this generative story can take, that is, the form with the least amount of assumptions, looks as follows:

1. For each position $i$ of a sequence, condition on the history $h_i$ and draw the $i$th word from the conditional probability distribution $P_{X|H=h_i}$. 
2. Stop generating if $X_i$ is the EOS token    

We say this procedure is very general because it is essentially just chain rule spelled out in English words, though here the order of enumeration is determined by the left-to-right order of tokens in an English sentence.


Here is an example for a sequence of length $m=3$:

$P_S(\langle x_1, x_2, x_3 \rangle) = P_{X|H}(x_1|\langle \rangle) P_{X|H}(x_2|\langle x_1 \rangle) P_{X|H}(x_3 | \langle x_1, x_2 \rangle)$

For our example sentence *He went to the store* this means:

\begin{align}
P_S(\langle \text{He, went, to, the, store, EOS} \rangle) &= P_{X|H}(\text{He}|\langle \rangle) \\
    &\times P_{X|H}(\text{went}|\langle \text{He} \rangle) \\
    &\times P_{X|H}(\text{to}|\langle \text{He}, \text{went} \rangle) \\
    &\times P_{X|H}(\text{the}|\langle \text{He},  \text{went}, \text{to} \rangle) \\
    &\times P_{X|H}(\text{store}|\langle \text{He},  \text{went}, \text{to}, \text{the} \rangle) \\
    &\times P_{X|H}(\text{EOS}|\langle \text{He},  \text{went}, \text{to}, \text{the}, \text{store} \rangle) 
\end{align}

* where with some abuse of notation we use the words themselves as outcomes instead of their corresponding indices. 




**Quiz 2**  Write down the general rule for the probability $P_S$ of a sentence $x_{1:m}$. Don't forget to use subscripts to indicate the precise random variable associated with every distribution (that is, for example, $P_S$ is correct while $P$ is wrong). 

<details>
    <summary><b>SOLUTION</b></summary>

$P_S(x_{1:m}) = \prod_{i=1}^{m}P_{X|H}(x_i|x_{<i})$

where $m$ is the length of the sentence.
    
</details>
    
---

The LM above is just an abstraction, not a concrete implementation, think of it as a general template for building models. 

A concrete model design needs to specify the conditional probability distributions in the model, this is known as the **parameterisation** of the model, and an algorithm for parameter estimation. 

In T1 we briefly discussed the unigram language model, we will revisit it, extend it to deal with unknown words, and then relax some of its strong independence assumptions.

# <a name="unigram"> Unigram LM


We start with the simplest language model: the idea is to forget the history completely, therefore making the strong assumption that words are drawn from the same distribution independently of thei preceding context (i.e., $X_i \perp X_{<i}$):

\begin{equation}
P_S(x_{1:m}) \overset{\text{ind.}}{\triangleq}  \prod_{i=1}^m P_X(x_i)
\end{equation}

For the parameterisation, we let $X$ follow a Categorical distribution with parameter $\theta_{1:v} \in \Delta^{v-1}$. 

Thus, the final <a name="eq-unigram-lm">unigram LM definition</a> is 

\begin{equation}
P_S(x_{1:m}|\theta_{1:v}) \triangleq \prod_{i=1}^m \text{Cat}(X=x_i|\theta_{1:v}) 
\end{equation}

If you need a recap of probability theory, check [this notebook](../R1/R1.ipynb).

**Quiz 3**  Complete the categorical probability mass function (pmf) and the conditions below:

$\text{Cat}(X=a|\theta_{1:v}) = $ *type the pfm*

where $\theta_{1:v} = \langle \theta_1, \ldots, \theta_v\rangle$ are the categorical parameters for which it must hold

1. *type condition 1*
2. *type condition 2*



<details>
    <summary><b>SOLUTION</b></summary>

$\text{Cat}(X=a|\theta_{1:v}) = \prod_{i=1}^v \theta_x^{\delta_{xa}}$

or 

$\text{Cat}(X=a|\theta_{1:v}) = \prod_{i=1}^v \theta_x^{[x=a]}$


or 

$\text{Cat}(X=a|\theta_{1:v}) = \theta_a$

where $\theta_1^v$ are the categorical parameters for which it must hold


1. $0 < \theta_x < 1$ for $x \in [1, v]$
2. $\sum_{x=1}^v \theta_x = 1$ 

</details>

---

***Example***

Consider the sentence *He went to the store*, its probability under the unigram LM is

$P_S(\langle \text{He, went, to, the, store, EOS} \rangle) = P_X(\text{He}) \times P_X(\text{went}) \times P_X(\text{to}) \times P_X(\text{the}) \times P_X(\text{store}) \times P_X(\text{EOS})$

which, as a function of the parameters of the model, evaluates to

$P_S(\langle \text{He, went, to, the, store, EOS} \rangle) = \theta_{\text{He}} \times \theta_{\text{went}} \times \theta_{\text{to}} \times \theta_{\text{the}} \times \theta_{\text{store}} \times \theta_{\text{EOS}}$

where again we use the words instead of their indices.

---

We now have a concrete instance of a language model:
* a set of independence assumptions (the unigram assumption)
* a parameterisation (we will use a single Categorical distribution)

If we are given the parameters $\theta_{1:v}$, we will be able to sample $S$ from this model, and we will be able to assess the probability the model assigns to any given sentence in its sample space. 

So, we just need to find a way to choose $\theta_{1:v}$. We could pick any vector in the probability simplex. Clearly, some probability vectors are better than others though, so we better find a procedure that enjoys some theoretical support.

We can turn to basic statistics for help, in particular, we can turn to maximum likelihood estimation.

## Maximum likelihood estimation

Suppose we are given a corpus $\mathcal D$ containing $K$ sentences

* each sentence is of the form $x_{1:m_k}^{(k)}$ for $k=1, \ldots, K$
* where $m_k$ is the length of the $k$th sentence

The MLE solution for the unigram LM is based on gathering counts and computing the relative frequency of word types:

\begin{equation}
\theta_x = \frac{\mathrm{count}_X(x)}{\sum_{x' \in \mathcal X}\mathrm{count}_X(x')} =  \frac{\overbrace{\sum_{k=1}^K \sum_{i=1}^{m_k} [x^{(k)}_i = x]}^{\text{count }X=x\text{ in }\mathcal D}}{\text{number of tokens in }\mathcal D}
\end{equation}

Note that the *number of tokens* is simply the sum of the length of the sentences $\sum_{k=1}^K m_k$.



## Implementation

Let's quickly implement the unigram LM (this implementation will differ slightly from the one we developed in T1, this way you get to appreciate some diversity of designs).

Let us use the PTB dataset (the NLTK version which is already segmented into sentences and tokens, [see T1](../T1/T1.ipynb)). We will estimate the categorical parameters and query the LM with some sentences to find out their probability.

Notes: 

1. Use python dict to store the parameters of our unigram distribution.
2. For English, we lowercase the data to collect better statistics (otherwise 'He' and 'he' would correspond to different words).

In [ ]:
import nltk

In [ ]:
from nltk.corpus import treebank as ptb 

First, we start by **loading and pre-processing data**. NLTK's PTB is already tokenized, so we only do case folding.

In [ ]:
prep_ptb = [[w.lower() for w in s] for s in ptb.sents()]
print("The PTB has {} sentences, here are the first few:".format(len(prep_ptb)))
for i, s in zip(range(5), prep_ptb):
    print(s)

We will need to pick an EOS symbol, as that symbol will be part of the unigram distribution. 
For convenient we will also pick a begin-of-sentence (BOS) symbol, wich will be useful later or when we look inot higher-order LMs. 

Let's first verify that our choices of BOS and EOS are not already in the corpus:

In [ ]:
from itertools import chain
ptb_original_vocab = set(chain(*prep_ptb))  # itertools.chain is very handy, check its documentation if you don't already know it

In [ ]:
assert "<s>" not in ptb_original_vocab, "Our choice of BOS symbol should not clash with an existing token"
assert "</s>" not in ptb_original_vocab, "Our choice of EOS symbol should not clash with an existing token"

Whenever we approach statistical learning, it is convenient to hold some data out of training. These data will help us adjust some model decisions throughout development and compare alternatives at the end. 

We will typically split the available data in 3 disjoint sets:

* *training* data: the portion we used for parameter estimation via MLE
* *development* data: the portion we used in the development phase to adjust some of our design choices
* *test* data: the portion that is kept heldout from training and development until we have decided on as few competing systems as possible, we use this portion to appreciate our model's potential for generalisation to novel data

An alternative to this fixed split, which is generally more robust, is *cross-validation*, however, for many NLP systems cross-validation adds a considerable amount of computation, so in our labs we will not be using it.

In [ ]:
import numpy as np

def split_corpus(sentences, ratio=0.8):
    """
    Randomly split a list of sentences into two sets according to the given ratio.
    
    Though the split is random, we fix the random seed of the random number generator, 
     this is good for reproducibility, it will allows us to compare your run to our runs.
    
    :param sentences: already tokenized sentences (list of strings)
    :returns: portion containing (ratio)*100% of the data, portion containing (1-ratio)*100% of the data
    """
    # This will guarantee that the permutation is the same every time (which is important for reproducibility)
    rng = np.random.RandomState(42)
    rng.permutation(5)
    indices = rng.permutation(len(sentences))
    n = int(indices.size * ratio)
    return [sentences[i] for i in indices[:n]], [sentences[i] for i in indices[n:]]

We will use 80% for training, 10% for development, and 10% for test.

In [ ]:
prep_training, prep_test = split_corpus(prep_ptb)
prep_dev, prep_test = split_corpus(prep_test, ratio=0.5)

print("Number of observations: training={} development={} test={}".format(len(prep_training), len(prep_dev), len(prep_test)))

Let's **count unigrams**, much like we did in T1:

In [ ]:
from collections import Counter


def count_unigrams(sentences, EOS="</s>"):
    """
    input: a generator of preprocessed sentences
        - a preprocessed sentence is a list of lowercased tokens
    output: 
        unigram_count: dictionary of frequency of each word
    """    
    unigram_counts = Counter()
    for sentence in sentences:
        unigram_counts.update(sentence + [EOS])
    
    ### Here is an alternative (less compact) way of counting unigrams
    #unigram_counts = defaultdict(int)
    #for sentence in sentence_stream:
    #    for token in sentence:
    #        unigram_counts[token] += 1  # frequency of each word
    
    return unigram_counts

In [ ]:
unigram_count_table =  count_unigrams(prep_training)
assert unigram_count_table['</s>'] == len(prep_training), "EOS should occur as many times as there are sentences in the corpus"

In [ ]:
# Let's test our procedure and check how many times 'cat', 'mat', and '</s>' happen in the PTB training corpus
print('unigram=cat count={}'.format(unigram_count_table['cat']))
print('unigram=mat count={}'.format(unigram_count_table['mat']))
print('unigram=</s> count={}'.format(unigram_count_table['</s>']))

Here is MLE as we saw in T1:

In [ ]:
import numpy as np


def unigram_mle(unigram_counts: Counter):
    """
    input: unigram_count: dictionary of frequency of each word
           
    output: unigram_prob: dictionary with the probabilty of each word 
            (parameters of the model)
    """
    
    total_count = sum(unigram_counts.values())
    unigram_probs = dict()
    
    for word, count in unigram_counts.items():
        unigram_probs[word] = float(count) / total_count            
        
    return unigram_probs

# Let's check the MLE parameters associated with 'cat' and 'mat' by querying their unigram probabilities
unigram_prob_table = unigram_mle(unigram_count_table)
assert all(0 <= p <= 1. for p in unigram_prob_table.values()), "Probabilities are between 0 and 1"
assert np.isclose(sum(unigram_prob_table.values()), 1., 0.0001), "The coordinates of a probability vector add up to 1.0"

And this is how we check the probability of a token, in a way that unknown words won't lead to a KeyError

In [ ]:
print('unigram=cat prob=%f' % unigram_prob_table.get('cat', 0.0))  # 0.0 is the default, returned in case the key is not in the dict
print('unigram=mat prob=%f' % unigram_prob_table.get('mat', 0.0))
print('unigram=ntmi prob=%f' % unigram_prob_table.get('ntmi', 0.0))
print('unigram=</s> prob=%f' % unigram_prob_table.get('</s>', 0.0))

In [ ]:
assert unigram_prob_table.get('ntmi', 0.0) == 0.0, "NLMI has not made it to the PTB"

In [ ]:
# Look how interesting: the probability of the EOS token is very close to the inverse of the average sequence length in the corpus
assert np.isclose(1./unigram_prob_table.get('</s>', 0.0), np.mean([len(s) for s in prep_training]), 0.1)

**Quiz 4** Why is it that $1/\theta_{\text{EOS}}$ is essentially equal to the average sequence length in the corpus?

<details>
    <summary><b>SOLUTION</b></summary>

The MLE estimate for EOS is

\begin{align}
\theta_{\text{EOS}} &= \frac{\mathrm{count}_X(\text{EOS})}{\text{number of tokens in corpus}}
\end{align}

the numerator is number of sentences (we appended EOS to the end of each and every sequence in the corpus), and `number of tokens in corpus` is the sum of sequence lengths, while `average sequence length` is precisely `sum of sequence lengths` divided by `number of sequences`.
    
</details>

---

**Quiz 5** Compute the log-probability of a sentence under the unigram LM.

In [ ]:
import numpy as np


def calculate_sentence_unigram_log_probability(sentence, word_probs, EOS="</s>", UNK="<unk>"):
    """
    input: list of words in a sentence
    word_probs: MLE paremeters
    output:
            sentence_probability_sum: log probability of the sentence
    """
    raise NotImplementedError

<details>
    <summary><b>SOLUTION</b></summary>

```python
    
import numpy as np


def calculate_sentence_unigram_log_probability(sentence, word_probs, EOS="</s>", UNK="<unk>"):
    """
    input: list of words in a sentence
    word_probs: MLE paremeters
    output:
            sentence_probability_sum: log probability of the sentence
    """
    sentence_log_probability = 0.
    # we first get the probability of unknown words
    #  which by default is 0. in case '<unk>' is not in the support
    unk_probability = word_probs.get(UNK, 0.)
    for word in sentence + [EOS]:
        # this will return `unk_probability` if the word is not in the support
        word_probability = word_probs.get(word, unk_probability)  
        # it is a sum of log pboabilities
        # we use np.log because it knows that log(0) is float('-inf')
        sentence_log_probability += np.log(word_probability)
    return sentence_log_probability  
    
```
    
</details>    
    
---    

In [ ]:
calculate_sentence_unigram_log_probability(['the', 'cat', 'sat', 'on', 'the', 'cat'], unigram_prob_table)

In [ ]:
assert np.isclose(
    calculate_sentence_unigram_log_probability(['the', 'cat', 'sat', 'on', 'the', 'cat'], unigram_prob_table), 
    -46.05, 0.01), "Unless you are not using the PTB or pre-processed it differently, I expected about -46.05"

We can assess the logarithm of the joint probability of subsets of our training data, for example, the first 10 sentences:

In [ ]:
sum(calculate_sentence_unigram_log_probability(s, unigram_prob_table) for i, s in zip(range(10), prep_training))

In [ ]:
assert np.isclose(
    sum(calculate_sentence_unigram_log_probability(s, unigram_prob_table) for i, s in zip(range(10), prep_training)), 
    -2110.23, 0.1), "Unless you are not using the PTB or pre-processed it differently, I expected about -2110.23"

## Unseen words

However, note that if we want the probability of sentences containing words that are not present in the training corpus, we will have an unpleasant surprise. 

For example: *NLMI is fun*


In [ ]:
calculate_sentence_unigram_log_probability(['ntmi', 'is', 'fun'], unigram_prob_table)

Heldout data will almost surely contain unseen words (remember Zipf's law?):

In [ ]:
sum(calculate_sentence_unigram_log_probability(s, unigram_prob_table) for i, s in zip(range(10), prep_dev))

**Quiz 6** Why did we get $-\infty$ above?

<details>
    <summary><b>SOLUTION</b></summary>

If a token is unknown it contributes a $0$ probability to the product in the joint probability $P_{X_{1:m}}(x_{1:m}|\boldsymbol \theta)$, which then becomes $0$, whose logarithm is $-\infty$.

Of course that would not happen if we had smoothed our language model.

</details>

---

# <a name="smooth"> Smoothing

Note that MLE will fail if we evaluate on sentences containing words that the model has never seen (at training). 
    
The words we haven't seen before are called unknown words, or **out of vocabulary** (OOV) words.
We will now map them to a special UNK symbol such as `<unk>`.
    
To keep the LM from assigning zero probability to these unseen words, we’ll have to steal some of the probability mass from some more frequent words and give it to words we've never seen.
This is called **smoothing** or **discounting**.

The simplest form of smoothing is called **Laplace smoothing**, whereby we add `<unk>` to the support of the distribution and then add one to all counts before we normalize them into probabilities. 
All the counts that used to be zero will now have a count of 1, the counts of 1 will be 2, and so on. 

We can also generalise it and add $\alpha > 0$ instead of $1$. Then for $X \sim \text{Cat}(\theta_{1:v})$ we get the MLE solution:

\begin{align}
\theta_x &= \frac{ \mathrm{count}_X(x) + \alpha}{\sum_{x' \in \mathcal X} (\mathrm{count}_X(x') + \alpha)} \\
    &= \frac{ \mathrm{count}_X(x) + \alpha}{\alpha v + \sum_{x' \in \mathcal X} \mathrm{count}_X(x')}
\end{align}
    
    
There are $v$ words in the vocabulary and each one was incremented by $\alpha$, we also need to adjust the denominator to take into account the extra $\alpha v$ observations.

**Quiz 7** Implement the smoothed MLE procedure for the unigram distribution

In [ ]:
def unigram_smoothed_mle(unigram_counts: Counter, alpha=1.0, UNK="<unk>"):
    """
    input: unigram_count: dictionary of frequency of each word
           
    output: unigram_prob: dictionary with the (smoothed) probabilty of each word 
            (parameters of the model)
    """
    raise NotImplementedError

<details>
    <summary><b>SOLUTION</b></summary>
    
```python
import numpy as np
from itertools import chain


def unigram_smoothed_mle(unigram_counts: Counter, alpha=1.0, UNK="<unk>"):
    """
    input: unigram_count: dictionary of frequency of each word
           
    output: unigram_prob: dictionary with the probabilty of each word 
            (parameters of the model)
    """
    assert UNK not in unigram_counts, "Choose an UNK symbol that is not currently part of the vocabulary"
    
    unigram_probs = dict()
    
    total_count = sum(unigram_counts.values())
    vocab_size = len(unigram_counts) + 1  # + 1 as we intend to consider UNK as part of the vocabulary
    denominator = total_count + alpha * vocab_size 
    
    for word, count in chain([(UNK, 0)], unigram_counts.items()):
        unigram_probs[word] = (float(count) + alpha) / denominator    
        
    return unigram_probs
```
    
</details>    
    
---

Let's smooth the unigram distribution and appreciate the effect on training data as well as heldout data:

In [ ]:
# Let's check the MLE parameters associated with 'cat' and 'mat' by querying their unigram probabilities
unigram_smoothed_prob_table = unigram_smoothed_mle(unigram_count_table)
assert all(0 <= p <= 1. for p in unigram_prob_table.values()), "Probabilities are between 0 and 1"
assert np.isclose(sum(unigram_smoothed_prob_table.values()), 1., 0.0001), "The coordinates of a probability vector add up to 1.0"

In [ ]:
assert unigram_smoothed_prob_table.get('ntmi', unigram_smoothed_prob_table["<unk>"]) > 0.0, "Smoothing!"
assert unigram_smoothed_prob_table.get('ntmi', unigram_smoothed_prob_table["<unk>"]) == unigram_smoothed_prob_table.get('mat', unigram_smoothed_prob_table["<unk>"]), "Smoothing!"

In [ ]:
log_prob_first10_before = sum(calculate_sentence_unigram_log_probability(s, unigram_prob_table) for i, s in zip(range(10), prep_training))
log_prob_first10_after  = sum(calculate_sentence_unigram_log_probability(s, unigram_smoothed_prob_table) for i, s in zip(range(10), prep_training))
assert log_prob_first10_after > log_prob_first10_before, "Smoothing generalise improves the probability of observed sentences"

In [ ]:
assert - np.infty < sum(calculate_sentence_unigram_log_probability(s, unigram_smoothed_prob_table) for i, s in zip(range(10), prep_dev)), "Smoothing should assign non-zero probability to heldout data"

# <a name="higher"> Higher order LMs

    
A unigram LM makes some rather unreasonable assumptions. Clearly, words in a sentence do depend on one another.

**Quiz 8** Can the unigram LM assign different probabilities to `what a nice day` and `day what nice a`?

<details>
    <summary><b>SOLUTION</b></summary>
    
    No really. The sentence contains the exact same tokens and they occur the exact same number of times.
    
    
</details>

---

If we want to capture dependencies in English sentences, we better go back to the general chain rule
    
\begin{equation}
P_S(x_{1:m}) = \prod_{i=1}^{m}P_{X|H}(x_i|x_{<i})
\end{equation}

and ask ourselves, why did we make such a strong independence assumption in the unigram LM and can we avoid it?


The deal is that with our current way of parameterising a Categorical distribution, we need to store  a $v$-dimensional parameter vector for every unique history in the training data. This type of parameterisation, where we have one parameter (a probability value) per outcome per context, is known as **tabular** (you can imagine storing the parameters in a table, each row is a context, each column is an outcome).
The more words we allow in the context, the more parameters we will have to estimate, and this tabular representation grows very quickly. Not only this costs a lot of parameters, most histories will appear very few times, a very long history will probably only appear once. Thus we will not be able to gather enough data to estimate the parameters of the LM reliably. 

We have at least two ways around this problem: we can revisit the model from the point of view of the (conditional) independencies we make, or we can change the way we parameterise Categorical distributions to make them more parameter efficient. This week we are going to do the former. Next week we begin to learn more about the latter. The combination of the two strategies lies at the core of much of the progress in modern NLP.

The core of the problem is the data sparsity due to long histories, then let's shorten the histry. But unlike the unigra LM, we won't discard the history entirely, we will just forget some of it instead. 

The $n$-gram LM assumes that $X_i$ is independent of all but the $n-1$ immediately preceding words. It uses Categorical distributions to model the distribution of $X_i | H_i=x_{i-n+1:i-1}$ in context. 

**Notation guide** As $x_{i-n+1:i-1}$ is a bit cumbersome to write, we will write $x_{[<i]_n}$ to denote the $i$th prefix shortened to $n-1$ words.


In its standard formulation, the $n$-gram LM uses tabular cpds, that is, it stores one Categorical distribution per unique history: $X | H=h \sim \text{Cat}(\theta_{1:v}^{(h)})$ with $\theta_{1:v}^{(h)} \in \Delta^{v-1}$ for each history $h$ (a sequence of $n-1$ tokens).

The joint probability the $n$-gram LM assigns to a sentence $x_{1:m}$ is

\begin{align}
P_S(x_{1:m}|\boldsymbol \theta) &\overset{cond.ind.}{\triangleq} \prod_{i=1}^m P_{X|H}(x_i|\underbrace{x_{[<i]_n}}_{=h_i}) \\
    &= \prod_{i=1}^m \text{Cat}(x_i|\theta_{1:v}^{(h_i)}) 
\end{align}

**Notation guide** We are using $\boldsymbol \theta$ to denote the collection of all prameters of all cpds.

The $n$-gram LM is what we call a **Markov model** of order $o=n-1$, the order indicates the length of the shortened history we condition on when drawing $X_i$.






**Quiz 9**  Write down the probability of the sentence 

    He went to the store
    
under a bigram language model. 

<details>
    <summary><b>SOLUTION</b></summary>

$P_S(\langle \text{He, went, to, the, store, EOS} \rangle) = P_X(\text{He}) \times P_X(\text{went}|\langle \text{He} \rangle) \times P_X(\text{to}| \langle \text{went} \rangle) \times P_X(\text{the}| \langle \text{to} \rangle) \times P_X(\text{store}|\langle \text{the} \rangle) \times P_X(\text{EOS}|\langle \text{store} \rangle)$

Tip: recall that *the* is a word while $\langle \text{the} \rangle$ is a sequence, and recall that though sentences in corpora don't usually come decorated with an EOS, our LMs treat them as if they did.
    
Trick: it's quite handy to imagine that we had $n-1$ occurences of a begin-of-sentence (BOS) symbol prior to the first token in the sentence, this gives us a fixed-size history across time steps: the first step would look like $P_X(\text{He}|\langle BOS \rangle)$.

</details>

---

### Parameter estimation

The Laplace-smoothed MLE solution for tabular cpds is simply

\begin{equation}
(4) \qquad \theta_x^{(h)} = \frac{\mathrm{count}_{HX}(h \circ \langle x \rangle) + \alpha}{\mathrm{count}_H(h) + \alpha v}
\end{equation}

where  $h \circ \langle x \rangle$ is the concatenation of history and word. 
    
    
**Notation guideline** When we use a superscript like $\theta_{1:v}^{(h)}$ we mean the probability vector selected by the history, or the probability vector that is specific to the cpd that conditions on the given history $h$. For example, in a bigram LM, we have one $v$-dimensional parameter vector for each one of the $v$ cpds in the model. That is, there is one cpd per word in our vocabulary, because each time we condition on a history $\langle w \rangle$, we get a different cpd, and each of these cpds is a discrete distribution over the entire vocabulary. 
    

**Quiz 10**  What is the space complexity of an $n$-gram LM? 

<details>
    <summary><b>SOLUTION</b></summary>

The $n$-gram LM requires a collection of cpds, each cpd takes the form $\text{Cat}(\theta_{1:v}^{(h)})$ where $h$ has size $n-1$ by definition. As $h$ is made of symbols in the vocabulary of the language, the maximum number of unique histories we can have is therefore $v^{n-1}$. This means we have an upperbound on the number of cpds. Each cpd is a discrete distribution over the vocabulary, that is, each $\theta_{1:v}^{(h)} \in \Delta^{v-1}$ is a $v$-dimensional probability vector. 
    
The space complexity of a general $n$-gram LM is therefore $\mathcal O(v^n)$.
    
</details>

---

**Quiz 11**  What is the time complexity to assess the joint probability of a sentence under an $n$-gram LM? 

<details>
    <summary><b>SOLUTION</b></summary>

To assess the joint probability of a sentence $x_{1:m}$, we need to scan over each position $i$ from $1$ to $m$ and look up the probability of $x_i$ in context. Assuming that lookup operations are efficient, i.e. they run in constant time $\mathcal O(1)$, the upperbound on runtime is linear in sentence length $\mathcal O(m)$.
    
    
</details>

---

<a name="ex1" style="color:red">**Exercise 1**</a> In this exercise you will build a general $n$-gram language model where $n \ge 1$. We provide you with a skeleton class on which to build. 


* Start by implementing the method `count_ngrams`, see the documentation of the method for specification. Tip: expand upon the procedure implemented in the function `count_unigrams` above; remember to handle BOS tokens and EOS tokens correctly. Use `<s>` for BOS token and `</s>` for EOS token.
* Now implement the method `solve_mle`, see the documentation of the method for specification.
* Finally, implement the `log_prob` method, see the documentation of the method for specification.

You should fill in a table like the following one:

In [ ]:
from tabulate import tabulate

headers = ['training', 'heldout', 'order', 'alpha', 'perplexity']
rows = [
    ['ptb', 'ptb-dev', 0, [0], None],    # this is a unigram LM
    ['ptb', 'ptb-dev', 0, [1],  None],    # this is a smoothed unigram LM
    ['ptb', 'ptb-dev', 1, [1, 0],  None],  # this is a smoothed bigram LM (with smoothed unigram counts)
    ['ptb', 'ptb-dev', 1, [1, 1],  None],  # this is a smoothed bigram LM (with smoothed unigram and bigram counts)
]
print("Log probability assigned to the sentences in the dev set")
print(tabulate(rows, headers=headers))

In [ ]:
from collections import defaultdict, Counter
import numpy as np
import sys


class LM:
    """
    This is an n-gram LM with generalised Laplace smoothing.    
    """
    
    def __init__(self, order, alpha=0.0, BOS="<s>", EOS="</s>", UNK="<unk>"):  
        """
        """
        self._order = order
        self._count_table = dict()
        self._prob_table = dict()
        self._vocab = set()
        self._BOS = BOS
        self._EOS = EOS
        self._UNK = UNK
        self._alpha = alpha   
        
        # in Laplace smoothing we always add '<unk>' to the vocabulary
        self._vocab.add(UNK)
        
    def order(self):
        return self._order
    
    def vocab_size(self):
        return len(self._vocab)
             
    def preprocess_history(self, history):
        """
        This function pre-process an arbitrary history to match the order of this language model.
        :param history: a sequence of words
        :return: a tuple containing exactly as many elements as the order of the model
            - if the input history is too short we pad it with <s> 
        """
        if len(history) == self._order:
            history = tuple(history)
        elif len(history) > self._order:
            length = len(history)            
            history = tuple(history[length - self._order: length])
        else:  # here the history is too short
            missing = self._order - len(history)
            history = tuple([self._BOS] * missing) + tuple(history)
        assert len(history) == self._order, "Make sure the history has a fixed-length, this simplifies implementation"
        return history
                
    def get_parameter(self, history, word):
        """
        This function returns the categorical parameter associated with a certain word given a certain history.
        :param history: a sequence of words (a tuple)
        :param word: a word (a str)
        :return: a float representing P(word|history)
        """
        raise NotImplementedError("I am an assignment, implement me!")
        
    def cpd_items(self, history):
        history = self.preprocess_history(history)
        # if the history is unseen we return an empty cpd
        return self._prob_table.get(history, dict()).items()
        
    def count_ngrams(self, data_stream):
        """
        This function should populate the attribute _count_table which should be understood as 
            - a python dict 
                - whose key is a history (a tuple of words)
                - and whose value is itself a python dict (or defaultdict)
                    - which maps a word (a string) to a count (an integer)
        
        This function will add counts to whatever counts are already stored in _count_table.
        
        :param data_stream: a generator as produced by `preprocess`
        """
        raise NotImplementedError("I am an assignment, implement me!")
                    
    def solve_mle(self):
        """
        This function should compute the attribute _prob_table which has the exact same structure as _count_table
         but stores probability values instead of counts. 
        It can be seen as the collection of cpds of our model, that is, _prob_table
            - maps a history (a tuple of words) to a dict where
                - a key is a word (that extends the history forming an ngram)
                - and the value is the probability P(word|history)                
                
        This function will replace whatever value _prob_table currently stores by the newly computed MLE solution.
        """
        raise NotImplementedError("I am an assignment, implement me!")
        
    def log_prob(self, sentence):
        """
        Compute the log probability of a sentence under this model. 
                
        input: 
            sentence: a sequence of tokens
        output:
            log probability
        """
        raise NotImplementedError("I am an assignment, implement me!") 

Let us play a bit with a unigram LM:

In [ ]:
# **DEBUG PTB**
unigram_lm = LM(order=0)
unigram_lm.count_ngrams(prep_training)
unigram_lm.solve_mle()

In [ ]:
print(unigram_lm.log_prob("the new rate will be payable feb. 15 .".split()))

In [ ]:
assert np.isclose(
    unigram_lm.log_prob("the new rate will be payable feb. 15 .".split()), 
    -65.36, 0.01), "Unless you changed the corpus, the preprocessing, or the random seed, I expected -65.36"

Clearly, our model always supports the training set

In [ ]:
for i, s in zip(range(5), prep_training):
    print('{:.2f} {}'.format(unigram_lm.log_prob(s), ' '.join(s)))

But the dev set may contain words/tokens unseen during training:

In [ ]:
for i, s in zip(range(5), prep_dev):
    print('{:.2f} {}'.format(unigram_lm.log_prob(s), ' '.join(s)))

Smoothing will circumvent that problem.

In [ ]:
# **DEBUG PTB**
smoothed_unigram_lm = LM(order=0, alpha=1.)
smoothed_unigram_lm.count_ngrams(prep_training)
smoothed_unigram_lm.solve_mle()

It's not necessarily the case that a smoothed LM will improve the probability of already observed sentences (after all we are stealing probability from some $n$-grams and moving it around to unseen outcomes).

In [ ]:
rows = []
for i, s in zip(range(5), prep_training):
    rows.append([i + 1, unigram_lm.log_prob(s), smoothed_unigram_lm.log_prob(s)])
print('Sentence from training data')    
print(tabulate(rows, headers=['i', 'unigram LM', 'smoothed unigram LM']))

But smoothing will deal with the problem of 0 probabilities for unobserved sentences:

In [ ]:
rows = []
for i, s in zip(range(5), prep_dev):
    rows.append([i + 1, unigram_lm.log_prob(s), smoothed_unigram_lm.log_prob(s)])
print('Sentence from development data')    
print(tabulate(rows, headers=['i', 'unigram LM', 'smoothed unigram LM']))

Increasing the LM order will generally improve the probability of already observed sentences:

In [ ]:
# **DEBUG PTB**
bigram_lm = LM(order=1)
bigram_lm.count_ngrams(prep_training)
bigram_lm.solve_mle()

In [ ]:
rows = []
for i, s in zip(range(5), prep_training):
    rows.append([i + 1, unigram_lm.log_prob(s), bigram_lm.log_prob(s)])
    assert bigram_lm.log_prob(s) > unigram_lm.log_prob(s), "Generally we expect the bigram LM to assign higher probabilities to training data than the unigram LM"
print('Sentence from training data')    
print(tabulate(rows, headers=['i', 'unigram LM', 'bigram LM']))

But generalising to novel data is even harder now: 

In [ ]:
rows = []
for i, s in zip(range(5), prep_dev):
    rows.append([i + 1, unigram_lm.log_prob(s), bigram_lm.log_prob(s)])
print('Sentence from development data')    
print(tabulate(rows, headers=['i', 'unigram LM', 'bigram LM']))

**Quiz 12** Why is it harder to generalise to novel data as the LM order increases?

<details>
    <summary><b>SOLUTION</b></summary>
   
Due to data sparsity. Without any smoothing, we will return 0 probability to unseen bigrams, the space of possible bigrams is much larger (i.e., $\{1, \ldots, v\} \times \{1, \ldots, v\}$) than the space of possible unigrams (i.e., $\{1, ..., v\}$) for the same amount of training data, thus it's more likely that we will encounter unseen bigrams after training.  
   
</details>

---

Smoothing overcomes 0s, but we will have other problems, as we shall see.

In [ ]:
# **DEBUG PTB**
smoothed_bigram_lm = LM(order=1, alpha=1.)
smoothed_bigram_lm.count_ngrams(prep_training)
smoothed_bigram_lm.solve_mle()

In [ ]:
rows = []
for i, s in zip(range(5), prep_dev):
    rows.append([i + 1, bigram_lm.log_prob(s), smoothed_bigram_lm.log_prob(s)])
print('Sentence from development data')    
print(tabulate(rows, headers=['i', 'bigram LM', 'smoothed bigram LM']))

**Quiz 13** If we compare a unigram LM (with Laplace smoothing) and a bigram LM (with Laplace smoothing for unigram and bigram distributions), we will see that smoothing is not doing a great job. While it does prevents $0$ probabilities for novel sentences, it sometimes makes the model a lot worse for already observed data. How would you explain this behaviour of smoothing applied to sparser distributions (such as the higher-order distributions)?

In [ ]:
rows = []
for i, s in zip(range(5), prep_training):
    rows.append(['training {}'.format(i + 1), smoothed_unigram_lm.log_prob(s), smoothed_bigram_lm.log_prob(s)])
for i, s in zip(range(5), prep_dev):
    rows.append(['dev {}'.format(i + 1), smoothed_unigram_lm.log_prob(s), smoothed_bigram_lm.log_prob(s)])    
print(tabulate(rows, headers=['i', 'smoothed unigram LM', 'smoothed bigram LM']))

<details>
    <summary><b>SOLUTION</b></summary>

Consider a conditional distribution $X|H=\langle w \rangle \sim \text{Cat}(\theta_{1:v}^{\langle w \rangle})$ for some single-word history. The parameters of this cpdf are estimated using bigram counts. Bigrams are sparser than unigrams, perhaps given $\langle w \rangle$, we only have $5$ observations in the training data. This means that we have only $5$ actual counts, against $\alpha v - 5$ fake counts (due to generalised Laplace smoothing). It's easy to see that unless $\alpha$ is quite small, unobserved outcomes will accummulate far more probabability mass than observed outcomes. 
    
To make smoothing works this constant $\alpha$ needs to be chosen rather carefully. 
    
</details>


---

Our advice is that you smooth low-order LMs (e.g., unigram LMs, maybe bigram LMs), but use other techniques for higher-order models, as we show next.

# <a name="inter"> Interpolation

Laplace smoothing deals with unseen words for a seen history, but it is not the best strategy for unseen histories. A simple idea is to use language model interpolation in order to obtain more robust statistics for our ngrams. 

We interpolate language models $\mathcal M_0, \ldots, \mathcal M_o$, where $\mathcal M_j$ is a Markov model of order $j$, to obtain an interpolated $(o+1)$-gram language model. For the interpolation we use coefficients $\lambda_0, \ldots, \lambda_o$ where

* $0 < \lambda_j < 1$
* $\sum_{j=0}^{o} \lambda_j = 1$

The probability of a sentence $x_{1:m}$ under the <a name="inter-snt-prob">interpolated model</a> is

\begin{equation}
(8) \qquad P_S(x_{1:m}|\mathcal M_0, \ldots, \mathcal M_o) = \prod_{i=1}^m P_{X|H}(x_i|x_{<i}, \mathcal M_0, \ldots, \mathcal M_o)
\end{equation}

where the <a name="inter-factor">interpolated factor is </a>

\begin{equation}
(9) \qquad P_{X|H}(x_i|x_{<i}, \mathcal M_0, \ldots, \mathcal M_{n-1}) = \sum_{j=0}^{o} \lambda_j \times P_{X|H}(x_i|x_{i-j:i-1}, \mathcal M_j)
\end{equation}

and $ P_{X|H}(x|h, \mathcal M_j)$ is the probability of the $(j+1)$-gram suffix of $h \circ \langle x \rangle$ under a model of order $j$.



For example, consider the sentence `here comes the sun`, for a $3$-gram LM (order $2$) we pad it `BOS BOS here comes the sun EOS` and compute interpolated factors:

\begin{align}
P_{X|H}(\text{here} \mid \langle \text{BOS, BOS} \rangle) &= \lambda_0 \times P_{X|H}(\text{here} \mid \langle \rangle, \mathcal M_0) \\
&+ \lambda_1 P_{X|H}(\text{here}\mid \langle \text{BOS} \rangle, \mathcal M_1) \\
&+ \lambda_2 P_{X|H}(\text{here} \mid \langle \text{BOS, BOS} \rangle, \mathcal M_2) \\
P_{X|H}(\text{comes}\mid \langle \text{BOS, here} \rangle) &= \lambda_0 \times P_{X|H}(\text{comes}\mid \langle \rangle, \mathcal M_0) \\
&+ \lambda_1 P_{X|H}(\text{comes}\mid\langle \text{here} \rangle,  \mathcal M_1) \\
&+ \lambda_2 P_{X|H}(\text{comes}\mid \langle \text{BOS, here} \rangle,  \mathcal M_2) \\
P_{X|H}(\text{the}\mid \langle \text{here, comes} \rangle) &= \lambda_0 \times P_{X|H}(\text{the}\mid\langle \rangle, \mathcal M_0) \\
&+ \lambda_1 P_{X|H}(\text{the}\mid \langle \text{comes} \rangle,  \mathcal M_1) \\
&+ \lambda_2 P_{X|H}(\text{the}\mid\langle \text{here, comes} \rangle,  \mathcal M_2) \\
P_{X|H}(\text{sun}\mid \langle \text{comes, the} \rangle) &= \lambda_0 \times P_{X|H}(\text{sun}\mid \langle \rangle, \mathcal M_0) \\
&+ \lambda_1 P_{X|H}(\text{sun}\mid \langle \text{the} \rangle,  \mathcal M_1) \\
&+ \lambda_2 P_{X|H}(\text{sun}\mid \langle \text{comes, the} \rangle,  \mathcal M_2)  \\
P_{X|H}(\text{EOS}\mid \langle \text{the, sun} \rangle) &= \lambda_0 \times P_{X|H}(\text{EOS}\mid \langle \rangle, \mathcal M_0) \\
&+ \lambda_1 P_{X|H}(\text{EOS}\mid \langle \text{sun} \rangle,  \mathcal M_1) \\
&+ \lambda_2 P_{X|H}(\text{EOS}\mid \langle \text{the, sun} \rangle,  \mathcal M_2) 
\end{align}

Then the probability of the sentence under the interpolation is 

\begin{align}
P_{S}(\langle \text{here, comes, the, sun, EOS}\rangle) 
&= P_{X|H}(\text{here} \mid \langle \text{BOS, BOS} \rangle) \\
&\times P_{X|H}(\text{comes}\mid \langle \text{BOS, here} \rangle)  \\
&\times P_{X|H}(\text{the}\mid \langle \text{here, comes} \rangle) \\
&\times P_{X|H}(\text{sun}\mid \langle \text{comes, the} \rangle) \\
&\times P_{X|H}(\text{EOS}\mid \langle \text{the, sun} \rangle)
\end{align}

Let's try and implement it

<a name="ex2" style="color:red">**Exercise 2**</a> Complete the class below which implements an interpolated language model.

1. Start by completing the method `get_parameter` which computes the interpolated factor $P_{X|H}$ ;
2. then complete the method `log_prob` which should use `get_parameter` to compute the log of the interpolated probability $P_{S|N}(x_{1:m})$



In [ ]:
class InterpolatedLM(LM):
    
    def __init__(self, lms, weights):
        """
        This class should interpolate language models, 
            there are certain conditions that they must hold.
            
        :params lms: a list of language models where the lms[i] should have order i
        :params weights: a list of positive weights that should sum to 1.0        
        """
        if not lms:
            raise ValueError('I need at least 1 language model')
        if not all(0 < w < 1 for w in weights) and sum(weights) != 1.0:
            raise ValueError('LM weights must sum to 1')
        # Let's check that we have the LMs we need
        for i, lm in enumerate(lms):
            if lm.order() != i:
                raise ValueError('Interpolation requires the ith LM to be of order i-1')
        self._max_order = lms[-1].order()  # the maximum order
        self._lms = lms
        self._weights = weights
        
    def order(self):
        return self._max_order
                
    def preprocess_history(self, history):
        raise NotImplementedError('You do not need to use or implement this method')
            
    def cpd_items(self, history):
        raise NotImplementedError('You do not need to use or implement this method')
        
    def count_ngrams(self, data_stream):
        raise NotImplementedError('You do not need to use or implement this method')
                    
    def solve_mle(self):
        raise NotImplementedError('You do not need to use or implement this method')
                
    def get_parameter(self, history, word):
        """
        This function should return the interpolated factor P(X=w|H=h) as defined in Equation (9) above.
    
        :param history: a sequence of words (a tuple)
        :param word: a word (a str)
        :return: a float representing P(word|history) in the interpolated model
        """       
        raise NotImplementedError("I am an assignment, implement me!")
    
    def log_prob(self, sentence):
        """
        Compute the log probability of a sentence under this model. 
                
        input: 
            sentence: a sequence of tokens
        output:
            log probability
        """
        raise NotImplementedError("I am an assignment, implement me!")

Let's train some LMs, and then interpolate them:

In [ ]:
# ***DEBUG***
lms = [
    LM(order=0, alpha=1.),        # unigram LM
    LM(order=1, alpha=0.),     # bigram LM
    LM(order=2, alpha=0.),  # trigram LM
]
# train our models
for lm in lms:
    lm.count_ngrams(prep_training)
    lm.solve_mle()

See that the bigram LM now does improve (on training and dev data), but improving the trigram LM on novel sentences (dev data) may require carefully tuning the interpolation weights.

In [ ]:
rows = []
for i, s in zip(range(5), prep_training):
    rows.append([
        'training {}'.format(i + 1), 
         InterpolatedLM(lms[0:1], [1.]).log_prob(s), 
         InterpolatedLM(lms[0:2], [1-1/10, 1/10]).log_prob(s),
         InterpolatedLM(lms[0:3], [1-(1/5+1/10), 1/5, 1/10]).log_prob(s)
    ])
for i, s in zip(range(5), prep_dev):
    rows.append([
        'dev {}'.format(i + 1), 
         InterpolatedLM(lms[0:1], [1.]).log_prob(s), 
         InterpolatedLM(lms[0:2], [1-1/10, 1/10]).log_prob(s),
         InterpolatedLM(lms[0:3], [1-(1/5+1/10), 1/5, 1/10]).log_prob(s)
    ])
print(tabulate(rows, headers=['i', 'LM1', 'LM2', 'LM3']))

Note that we start with a unigram LM alone, we then try interpolating a unigram LM and a bigram LM and improve perplexity considerably. 

Curiously, further interpolating trigram and fourgram LM does not really help much. This has again to do with data sparsity: our training corpus is not very large and therefore most 3-grams and 4-grams are quite rare. If there's little overlap between training and test in terms of 4-grams, the 4-gram terms in the interpolation will be mostly dominated by Laplace smoothing.

# <a name="eval">  Evaluation

The way to evaluate the performance of an LM is to test into a final application. In other words, how much the final score of the application improves. This is called *extrinsic* evaluation. Also, we can test our LM independently from an application, this is called *intrinsinc* evaluation. In this course, we are going to study the intrinsic evaluation of the LM.

We generally have access to 3 datasets: 

* Training is used for estimating $\boldsymbol \theta$ (we use boldface to indicate a collection of parameters).
* Develpment is used to make choices across models.
* Test is used for measuring the accuracy of the model.
   
In n-gram LM the evaluation is defined by the **likelihood** of the model with respect to the test dataset.
The likelihood of the parameters $\theta$ over the test dataset is the probability that the model assigns to the dataset.

We assume the test data $\mathcal T$ consits of $K$ independent sentences each denoted $x_{1:m_k}^{(k)}$ 

$P(\mathcal T| \boldsymbol \theta) = \prod_{k=1}^K P_S(x_{1:m_k}^{(k)}|\boldsymbol \theta)$

Or in form of the log-probability:

$\log P(\mathcal T| \theta) = \sum_{k=1}^K \log P_{S}(x_{1:m_k}^{(k)}| \theta)$

Then define the log-likelihood as follows:

$\mathcal L_{\mathcal T}(\boldsymbol \theta) = \sum_{k=1}^K \log P_{S}(x_{1:m_k}^{(k)}| \boldsymbol \theta)$


Then the model that assings the higher $\mathcal L$ given the test set is the one that best fits the data. In other words, given two probabilistic models, the one that assigns a higher probability to the test data is taken as intrinsically better. One detail we need to abstract away from is differences in factorisation of the models which may cause their likelihoods not to be comparable, but for that we will define *perplexity* below. 

The log likelihood is used because the probability of a particular sentence according to the LM can be a very small number, and the product of these small numbers can become even smaller, and it will cause numerical
precision problems. 


**Perplexity** of a language model on a test set is the inverse probability of the test set, normalized
by the number of tokens. Perplexity is a notion of average branching factor, thus a LM with low perplexity can be thought of as a *less confused* LM. That is, each time it introduces a word given some history it picks from a reduced subset of the entire vocabulary (in other words, it is more certain of how to continue). 

If a dataset contains $t$ tokens where $t = \sum_{k=1}^K m_k$, then the perplexity of the model given the test set is

\begin{equation}
(6) \qquad \text{PP}_{\mathcal T}(\boldsymbol \theta) = \left( \prod_{k=1}^K P_{S}(x_{1:m_k}^{(k)}|\boldsymbol \theta) \right)^{-1/t}
\end{equation}


It's again convenient to use log and define log-perplexity

\begin{equation}
(7) \qquad \log \text{PP}_{\mathcal T}(\boldsymbol \theta) = - \frac{1}{t} \sum_{k=1}^K \log P_{S}(x_{1:m_k}^{(k)}|\boldsymbol \theta) 
\end{equation}

You can compare models in terms of the log-perplexity given you base its computation on the same test data. The lower the perplexity, the better the model is.

Comparisons in terms of perplexity are only fair if the models have the same vocabulary.


<a name="ex3" style="color:red">**Exercise 3**</a> Implement the log-perplexity function below. See the function documentation for specifications.


In [ ]:
def log_perplexity(data_stream, lm):
    """
    Calculates the perplexity of the given text.
    This is simply 2 ** cross-entropy for the text.
    
    This function can make use of `lm.order()`, `lm.get_parameter()`, and `lm.log_prob()` 

    :param data_stream: generator of sentences (each sentence is a list of words)
    :param lm: an instance of the class LM
    """
    raise NotImplementedError("I am an assignment, implement me!")

In [ ]:
def perplexity(data_stream, lm):
    return np.exp(log_perplexity(data_stream, lm))

In [ ]:
# **DEBUG**
two_sentences_data = [
    "The commission did not do much .".lower().split(),
    "They discussed the matter again .".lower().split()
]

In [ ]:
ppl = perplexity(two_sentences_data, InterpolatedLM(lms[0:1], [1]))
print(ppl)

In [ ]:
perplexity(two_sentences_data, InterpolatedLM(lms[0:2], [1-1/10, 1/10]))

<a name="ex4" style="color:red">**Exercise 4**</a> Train a unigram LM, a bigram LM, and a trigram LM. Use smoothing for at least one of your models (e.g., the unigram LM). Compare them on heldout data in terms of perplexity. Make sure to include at least one interpolation model in the comparison, as shown in the table below. 

In [ ]:
from tabulate import tabulate
rows = [
    ('smoothed unigram', '?'),
    ('bigram', '?'),    
    ('trigram', '?'),
    ('interpolate 1-2', '?'),
    ('interpolate 1-3', '?'),
]
print(tabulate(rows, headers=['LM', 'heldout ppl']))    

# <a name="hparams">  Tuning hyperparameters


In this part we are going to use NLTK in order to play with a good discounting technique for LMs.

Kneser-Ney smoothing  is explained in the [textbook (Section 3.5)](https://web.stanford.edu/~jurafsky/slp3/3.pdf). Have a look at the section to get a bit of the intuition, but don't worry about the algorithmic details as we will not assess your understanding of that in this course. 
   

We will be working with the package [nltk.lm](http://www.nltk.org/api/nltk.lm.html) and in particular the class `KneserNeyInterpolated` which implements the parameter estimation algorithm described in the textbook. This  technique has a hyperparameter known as the *discounting factor* which we can choose. There's no automatic way to select this factor, but we can perform a *grid search*, that is, we can choose a range of plausible values between $0$ and $1$ and try those out. With the help of a development set we can pick the LM that performs the best intrinsically, and then use that LM to score future unseen test sets.

<a name="ex5" style="color:red">**Exercise 5**</a>

* Use NLTK's `KneserNeyInterpolated` class to estimate bigram and trigram LMs
* Perform a grid search for the discout factor and use dev-set perplexity to select the best bigram LM and the best trigram LM
* For each model, plot discount factor vs perplexity
* Finally, use the test set to compare your best LMs in terms of perplexity

**Warning** NLTK treats the $n$ in an $n$-gram language model as the *order* of that LM. This is not very starndard terminology with respect to literature on Markov models, but that's okay. Just be careful when creating language models using NLTK code. If you use our helper function `fit_and_eval` you don't need to worry about this.

In [ ]:
import nltk
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
from tabulate import tabulate
import numpy as np

In [ ]:
def fit_and_eval(training, heldout, longest, discount):
    """
    Train a KneserNey interpolated n-gram LM on `training` and evaluate its perplexity on `heldout`.
    :param training: a collection of tokenized sentences (a list of list of strings)
    :param heldout: a collection of tokenized sentences used for evaluation (list of list of strings)
    :param longest: this is the length of the longest ngram, in other words, this is the $n$ in $n$-gram LMs (as defined in class)
    :param discount: a number between 0 and 1 for smoothing
    :return LM object, and model's perplexity on heldout data
    """
    assert 0 <= discount <=1, "Discount is between 0 and 1"
    assert longest > 0, "Longest here is the n in n-gram LM"
    lm = KneserNeyInterpolated(longest, discount=discount)
    train, vocab = padded_everygram_pipeline(longest, training)
    lm.fit(train, vocab)
    return lm, lm.perplexity(heldout)